In [2]:
import torch
import os
from helpers.VAE.modelLoader import load_variational_mgt_model
import IPython.display as ipd
# import sys
import pickle
import bz2
import plotly.express as px
import plotly.graph_objects as go

from hvo_sequence.hvo_seq import HVO_Sequence
from hvo_sequence.drum_mappings import ROLAND_REDUCED_MAPPING

import numpy as np
import pandas as pd

from bokeh.io import output_notebook
output_notebook() 

Could not import fluidsynth. AUDIO rendering will not work.


Loading BokehJS ...

In [3]:
# Utility functions
def plot(hvo_seq_sample):
  """ Plots the piano roll of the sequence stored in the hvo_sequence object
  """
  hvo_seq_sample.piano_roll(show_figure=True)

# Loading, inspecting and making predictions with the Guitar2Drum Dataset

The data is stored as a dictionary inside a <code>bz2pickle</code> file. It should be decompressed and read accordingly:

In [4]:
with bz2.BZ2File('data/i2dgd/guitar2drum.bz2pickle', 'rb') as file:
    data = pickle.load(file)

The dictionary has the following structure:

In [5]:
for key in data.keys():
    print(key)
    for item in data[key].keys():
        print(f"\t{item}")

train
	inputs
	outputs
	outputs_hvo_seqs
	filenames
	style_primary
test
	inputs
	outputs
	outputs_hvo_seqs
	filenames
	style_primary


- **inputs:** List of HVO information for each input sequence.
- **outputs:** List of HVO information for each output sequence.
- **outputs_hvo_seqs:** List of HVO Sequence objects for each output sequence.
- **filenames:** List of Lakh_Ids of the original song transcriptions for each output sequence.
- **style_primary** List of style/genre for each input/output sequences. 

We can obtain the HVO information for the paring of sequences stored at position `idx`:

In [6]:
idx = 0

print(f"Guitar HVO\tType {type(data['train']['inputs'][idx])}\tDimensions {data['train']['inputs'][idx].shape}")
print(f"Drum HVO\tType {type(data['train']['outputs'][idx])}\tDimensions {data['train']['outputs'][idx].shape}")
print(f"Drum HVO Object\tType {data['train']['outputs_hvo_seqs'][idx]}")
print(f"Song Lakh Id\tType {type(data['train']['filenames'][idx])}: {data['train']['filenames'][idx]} ")
print(f"Style\tType {type(data['train']['style_primary'][idx])}: {data['train']['style_primary'][idx]} ")

# 

Guitar HVO	Type <class 'numpy.ndarray'>	Dimensions (32, 3)
Drum HVO	Type <class 'numpy.ndarray'>	Dimensions (32, 3)
Drum HVO Object	Type <hvo_sequence.hvo_seq.HVO_Sequence object at 0x000001908E7E4CA0>
Song Lakh Id	Type <class 'str'>: 66536dc07ecb2dac3fdf287f64acfe1e 
Style	Type <class 'str'>: Rock 


For convenience one can separate the data into smaller dictionaries:

In [7]:
inputs = data['test']['inputs']
outputs = data['test']['outputs']
outputs_hvos = data['test']['outputs_hvo_seqs']

## Sample predictions

We provide the file `predictions.bz2pickle` stored inside the models folder where we have stored a round of predictions for the test set for every presented model.

In [8]:
with bz2.BZ2File('models/predictions.bz2pickle', 'rb') as file:
    predictions = pickle.load(file)

The following script generates a plot for the predictions of models 5 and 8.

In [9]:
from random import randint

idx = randint(0, len(outputs_hvos)-1)

# Guitar Groove HVO
input_hvo = outputs_hvos[idx].copy_zero()
input_hvo.hvo = inputs[idx]

# Original Drum Groove HVO
output_hvo = outputs_hvos[idx]

# Predicted Drum Grooves HVOs
pred1_hvo = outputs_hvos[idx].copy_zero()
pred2_hvo = outputs_hvos[idx].copy_zero()
pred1_hvo.hvo = predictions['Model 5'][idx]
pred2_hvo.hvo = predictions['Model 8'][idx]

print("Guitar Groove")
plot(input_hvo)
print("Original Drum Groove")
plot(output_hvo)
print("Model 5 Generated Drum Groove")
plot(pred1_hvo)
print("Model 8 Generated Drum Groove")
plot(pred2_hvo)

Guitar Groove


Original Drum Groove


Model 5 Generated Drum Groove


Model 8 Generated Drum Groove
